In [29]:
import requests
import json
import pandas as pd

# API secret key
from secret import KEY_SECRET
from secret import KEY_ID

In [4]:
url = "https://data.cityofchicago.org/resource/ijzp-q8t2.json?$limit=10"
headers = {'Accept': 'application/json', 'X-App-Token': APP_TOKEN}
resp = requests.get(url,headers=headers)
test = json.loads(resp.text)
test2 = pd.DataFrame(test)
test2

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,longitude,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu,:@computed_region_rpca_8um6,:@computed_region_d9mm_jgwp,:@computed_region_d3ds_rm58
0,12387972,JE257757,2021-06-07T23:52:00.000,063XX S RACINE AVE,0460,BATTERY,SIMPLE,SIDEWALK,False,True,...,-87.654491644,"{'latitude': '41.779136938', 'longitude': '-87...",19,22257,65,21,2,23,17,203
1,12387967,JE257762,2021-06-07T23:45:00.000,067XX S PAULINA ST,0580,STALKING,SIMPLE,RESIDENCE,False,False,...,-87.666427817,"{'latitude': '41.771484402', 'longitude': '-87...",44,22257,65,283,2,23,17,204
2,12387906,JE257763,2021-06-07T23:45:00.000,059XX W MADISON ST,0460,BATTERY,SIMPLE,STREET,False,False,...,-87.772094898,"{'latitude': '41.880129791', 'longitude': '-87...",52,22216,26,618,7,32,25,99
3,12388248,JE258171,2021-06-07T23:42:00.000,063XX S KENWOOD AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,...,-87.593052128,"{'latitude': '41.779715221', 'longitude': '-87...",53,22260,9,27,4,60,18,273
4,12387968,JE257751,2021-06-07T23:30:00.000,009XX E MARQUETTE RD,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,APARTMENT,False,False,...,-87.602722558,"{'latitude': '41.774993832', 'longitude': '-87...",32,22260,9,347,33,60,18,206
5,12388398,JE258381,2021-06-07T23:30:00.000,014XX W 91ST ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,-87.660116624,"{'latitude': '41.728554085', 'longitude': '-87...",18,21554,72,195,13,59,9,244
6,12387888,JE257747,2021-06-07T23:29:00.000,018XX S RIDGEWAY AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,-87.718807245,"{'latitude': '41.855787154', 'longitude': '-87...",36,21569,30,800,14,57,21,156
7,12387903,JE257746,2021-06-07T23:24:00.000,086XX S STATE ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,GAS STATION,True,False,...,-87.624170849,"{'latitude': '41.736886394', 'longitude': '-87...",31,21554,40,428,32,61,20,241
8,12387941,JE257756,2021-06-07T23:23:00.000,012XX N STATE PKWY,0560,ASSAULT,SIMPLE,APARTMENT,False,False,...,-87.62869647,"{'latitude': '41.90447393', 'longitude': '-87....",51,14926,37,308,34,54,14,197
9,12387932,JE257732,2021-06-07T23:08:00.000,016XX W 77TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,-87.665117508,"{'latitude': '41.753973639', 'longitude': '-87...",17,21554,70,573,31,59,20,231
